## 1.3 신경망의 학습
학습되지 않은 신경망은 '좋은 추론'을 해낼 수 없다.  
그래서 학습을 먼저 수행하고, 그 학습도니 매개변수를 이용해 추론을 수행하는 것이 일반적이다.  
추론이란 앞 절에서 본 것 같은 다중 클래스 분류등 문제에 답을 구하는 작업이다.  
신경망의 학습은 최적의 매개 변수 값을 찾는 작업입니다.

## 1.3.1 손실 함수
신경망 학습에는 학습이 얼마나 잘 되고있는지를 알기 위한 '척도'가 필요로 한다.  
일반적으로 학습 단계의 특정 시점에서 신경망의 성능을 나타내는 척도로 손실(Loss)을 사용한다.  
손실은 학습 데이터(학습시 주어진 정답 데이터)와 신경망이 예측한 결과를 비교하여 예측이 얼마나 나쁜가를 산출한 단일 값(스칼라)이다.  

신경망의 손실은 손실 함수(Loss function)를 사용해 구한다.  
다중 클래스 분류(multi-class classification) 신경망에서는 손실 함수로 흔히 교차 엔트로피 오차(Cross Entropy Error)를 사용한다.  
교차 엔트로피 오차는 신경망이 출력하는 각 클래스의 확률과 정답 레이블을 이용해 구할 수 있다.  

#### SoftMax function
소프트맥스 함수의 출력의 각 원소는 0.0이상 1.0이하의 실수이다.    
그 원소들을 모두 더하게 되면 1.0이 나오게 되는데 이를 '확률'로 해석할 수 있다.  
$y_k = \frac{\exp(a_k)}{\sum_{i=1}^{n} \exp(a_i)}$

#### Cross Entropy Error
소프트 맥스의 출력을 받아 사용한다.  
$E = -\sum_{k} t_k \log y_k$  
이 때, $t_k$는 정답 레이블을 나타내고 정답 레이블은  
$t = [0, 0, 1]$  
원핫 벡터로 표현하여, 1이 아닌 값은 실질적으로 사용될 수 없게 된다.

#### Cross Entropy Error Mini Batch
$E = -\frac{1}{N} \sum_{n} \sum_{k} t_{nk} \log y_{nk}$  

$y_nk$는 신경망의 출력이고, $t_nk$는 정답 레이블이다.  

## 1.3.2 미분과 기울기
신경망의 학습 목표는 손실을 최소화하는 매개변수를 찾는 것이다.  
이때 중요한 것이 '미분'과 '기울기'이다.  
<br>
$L = f(x)$  

미분은 스칼라 값 뿐 아니라 다변수도 미분할 수 있다.  
<br>
$\frac{\partial L}{\partial \mathbf{x}} = \left[ \frac{\partial L}{\partial x_1}, \frac{\partial L}{\partial x_2}, \ldots, \frac{\partial L}{\partial x_n} \right]$  

각 원소에 대한 미분을 계산한 것이 기울기(gradient)이다.  

## 1.3.3 연쇄법칙
연쇄 법칙이란 합성함수에 대한 미분의 법칙이다.  
많은 함수를 연결하더라도, 그 미분은 개별 함수의 미분들을 이용하여 구분할 수 있다.  
국소적인 미분을 계산할 수 있다면 그 값들을 곱해서 전체의 미분을 구할 수 있다.  
신경망은 여러 함수가 연결된 것이라 표현할 수 있는데, 오차역전파법은 그 여러 함수(신경망)에 대해 연쇄 법칙을 효율적으로 적용하여 기울기를 구해낼 수 있다.  

## 1.3.4 계산 그래프
계산 그래프(computational graph)는 계산 과정을 그래프로 나타낸 것이며, **노드**(node)와 **엣지**(edge)로 표현된다. 노드는 연산을 정의하며, 엣지는 데이터가 흘러가는 방향을 나타낸다.

#### 덧셈 노드
덧셈 노드의 역전파는 입력값을 그대로 흘려보낸다.  
<br>
#### 곱셈 노드
곱셈 노드의 역전파는 순전파 입력을 서로 바꾼 값을 곱한다.  
<br>
#### 분기 노드
분기 노드는 선이 두 개로 나뉘면서, 같은 값이 복제 된다.  
분기노드의 역전파는 상류에서 온 기울기들의 합이 된다.  
<br>
#### Repeat 노드
분기 노드를 일반화 하면 $N$개로 분기 되는데, 이를 Repeat 노드라고 한다.  
<br>
#### SUM 노드
SUM 노드는 범용 덧셈 노드이다.  
```
# Sum 노드 역전파 예제
import numpy as np

D, N = 8, 7
x = np.random.randn(N, D)  # 입력
y = np.sum(x, axis=0, keepdims=True)  # 순전파

print(f'x.shape: {x.shape}')
print(f'y.shape: {y.shape}')
```
SUM 노드와 Repeat 노드는 서로 반대 관계이다.  
예시로 SUM노드의 순전파가 Repeat 노드의 역전파가 되며, SUM노드의 역전파가 Repeat노드의 순전파가 된다는 뜻이다.
<br>
#### MatMul노드
책에서 행렬의 곱셈을 MatMul이라 표현하고, 'Matrix Multiply'의 약자로 표현한다.  

## 1.3.5 기울기 도축과 역전파 구현
#### Sigmoid 계층

In [3]:
import numpy as np
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

#### Affine 계층

In [4]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

#### Softmax with Loss 계층

In [6]:
import sys
sys.path.append('..')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
# from common.functions import softmax, cross_entropy_error # git repo에 있음

class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None  # softmax의 출력
        self.t = None  # 정답 레이블

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        # 정답 레이블이 원핫 벡터일 경우 정답의 인덱스로 변환
        # cross_entropy_error()에 이미 있어서 굳이 필요 없을 듯
        if self.t.size == self.y.size:
            self.t = self.t. argmax(axis=1)

        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx /= batch_size

        return dx

## 1.3.6 가중치 갱신
오차역전파법으로 기울기를 구한 뒤, 그 기울기를 사용해 신경망의 매개변수를 갱신한다.  
이 때 신경망의 학습 순서는 아래와 같다.  
**1. 미니배치**  
훈련 데이터 중에서 무작위로 다수의 데이터를 골라낸다.  
**2. 기울기 계산**  
오차역전파법으로 각 가중치 매개변수에 개한 손실 함수의 기울기를 구한다.  
**3. 매개변수 계산**  
기울기를 사용하여 가중치 매개변수를 갱신한다.  
**4. 반복**  
1~3단계를 필요한 만큼 반복한다.  
<br>
매개변수를 그 기울기와 반대 방향으로 갱신하면 손실을 줄일 수 있다. 이를 경사하강법(Gradient Descent)라고 한다.  
방법 중 가장 단순한 방법으로 확률적경사하강법(Stochastic Gradient Descent, SDG)가 있다.  
그 뜻으로는 확률적(Stochastic)은 무작위로 선택된 데이터(미니배치)에 대한 기울기를 이용한다는 뜻을 가지고 있다.  

In [9]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr  # learning rate

    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]